# 00 - Connecting to the Azure OpenAI API to explore prompt engineering

In this lab, we will connect to our Azure OpenAI API for practicing prompt engineering. For each exercise, you'll get some input text and then an expected completion. Your task is to write the prompt to achieve the expected completion.

But first we will run a few test API calls to make sure everything works.

## Setup

First, we need to retrieve values from the `.env` file which we will use to make calls to the Azure OpenAI API.

In [1]:
import json
import requests
import os
from dotenv import load_dotenv

# Load environment variables
if load_dotenv():
    print("Found OpenAI API Base Endpoint: " + os.getenv("OPENAI_API_BASE"))
else: 
    print("OpenAI API Base Endpoint not found. Have you configured the .env file?")
    
API_KEY = os.getenv("OPENAI_API_KEY")
# This version of the API is needed to properly retrieve the list of model deployments.
API_VERSION = "2023-03-15-preview"
RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE")

Found OpenAI API Base Endpoint: https://aoai-jkordick.openai.azure.com/


Next, we'll set the endpoint that we're going to call. This endpoint will return a list of available model deployments.

In [2]:
url = RESOURCE_ENDPOINT + "/openai/deployments?api-version=" + API_VERSION

print (url)

https://aoai-jkordick.openai.azure.com//openai/deployments?api-version=2023-03-15-preview


Above you will see the full URL that we are going to be calling.

## Get the list of available Model Deployments

Now let's call that Azure OpenAI endpoint and see what it returns. We pass the API key in the HTTP header.

In [3]:
r = requests.get(url, headers={"api-key": API_KEY})

print(r.text)

{
  "data": [
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "gpt-35-turbo-16k",
      "owner": "organization-owner",
      "id": "gpt-35-turbo-16k-240k",
      "status": "succeeded",
      "created_at": 1695647543,
      "updated_at": 1695647543,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-embedding-ada-002",
      "owner": "organization-owner",
      "id": "text-embedding-ada-002-240k",
      "status": "succeeded",
      "created_at": 1695647595,
      "updated_at": 1695647595,
      "object": "deployment"
    }
  ],
  "object": "list"
}


Above, there should now be a JSON formatted list of model deployments. The output will contain one or more sections similar to the following

```
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "gpt-35-turbo",
      "owner": "organization-owner",
      "id": "gpt35turbo",
      "status": "succeeded",
      "created_at": 1684150536,
      "updated_at": 1684150536,
      "object": "deployment"
    }
```

In the output, you'll see the **Model Name** as the `model` value and the **Deployment Name** as the `id` value. We'll be using an `id` value for the next section.

## Send our first prompt to Azure OpenAI using the API

Next, let's call the Azure OpenAI API with a prompt. To do this, we'll need the `id` of our completion model deployment.

We've already setup this value in the `.env` file, but you should see a matching value in the list that was output above.

In [4]:
# Grab API Version in the .env file.
API_VERSION = os.getenv("OPENAI_API_VERSION")
DEPLOYMENT_ID = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")

As before, we'll construct a URL to call. This time, we'll also be creating a payload.

In [5]:
url = RESOURCE_ENDPOINT + "/openai/deployments/" + DEPLOYMENT_ID + "/chat/completions?api-version=" + API_VERSION

print(url)

https://aoai-jkordick.openai.azure.com//openai/deployments/gpt-35-turbo-16k-240k/chat/completions?api-version=2023-05-15


Again, you will see the full URL that we are going to call. This URL will use the specified deployment to call the **chat completions** API.

Next, we will call the Azure OpenAI API using the URL above. Just like last time, we pass the API key in the HTTP header. We also send a JSON formatted body as part of the request which contains the *prompt* that we want to use to get a response from the OpenAI model. In this case, our prompt is "Once upon a time", which should cause the model to complete the prompt by generating a story.

In [12]:
content = "Once upon a time"

r = requests.post(url, headers={"api-key": API_KEY}, json={"messages":[{"role": "assistant", "content": content}]})

print(json.dumps(r.json(), indent=2))

{
  "id": "chatcmpl-82gSwUD94gNKZcnh301MByf19Qf5K",
  "object": "chat.completion",
  "created": 1695650586,
  "model": "gpt-35-turbo-16k",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "in a small village, there lived a young girl named Lily. She had long, flowing red hair and bright blue eyes. Lily was known for her kindness and helpful nature.\n\nOne day, as Lily was walking through the village, she noticed an elderly woman struggling to carry a heavy bag of groceries. Without hesitation, Lily hurried over and offered her assistance. The woman gratefully accepted, and they walked together to the woman's house.\n\nInside, the woman introduced herself as Mrs. Jones. She lived alone and found it difficult to manage certain tasks due to her age. Lily spent the afternoon helping Mrs. Jones with various chores around the house, from cleaning to organizing.\n\nAs the days went by, Lily and Mrs. Jones 

As before, the result of the API call will be JSON data similar to the below example.

```
{
  "id": "chatcmpl-7wVxE2LNxaY6ZoxcWAmhiyrqaLZgf",
  "object": "chat.completion",
  "created": 1694180212,
  "model": "gpt-35-turbo",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": ", there was a magical kingdom called Tildor ... )))"
      }
    }
  ],
  "usage": {
    "completion_tokens": 366,
    "prompt_tokens": 13,
    "total_tokens": 379
  }
}
```

Here's some more information about some of the data contained in that response.

Key | Description
--- | ---
`model` | The model that was used to generate the response to the prompt
`content` | This is the response that was generated by the OpenAI model. Note that the response example above was edited to remove most of the output to make it easier to read
`finish_reason` | This is the reason that the model stopped generating the response. In this case, `stop` indicates the model determined it had fully completed the response without hitting any limits
`completion_tokens` | The number of tokens that were used in generating the response
`prompt_tokens` | The number of tokens that were consumed by the prompt
`total_tokens` | The total number of tokens that were consumed by the request (`prompt_tokens` + `completion_tokens`)

Now to our first exercise!

You can use the following code snippet where you can set your own prompt into the content variable.

We adjusted the `print` command to make the handling a bit easier.

In [22]:
content = 'Hello World'

multiline_content = 'This is a multiline string. \
It can span multiple lines and contain line breaks.'

response = requests.post(url, headers={"api-key": API_KEY}, json={"messages":[{"role": "assistant", "content": content}]})

data = response.json()

print(data['choices'][0]['message']['content'])

Lorem ipsum dolor sit amet, consectetur adipiscing elit.
Sed sed justo libero.
Vivamus viverra mi sit amet purus rutrum, sit amet tincidunt nibh porttitor.
Phasellus aliquam condimentum nibh, ac dignissim ante placerat ac.
Donec hendrerit, justo at vestibulum aliquam, lectus nunc bibendum augue, at fringilla neque diam vel ligula.
Vestibulum ante ipsum primis in faucibus orci luctus et ultrices posuere cubilia curae; Fusce dictum ex vel libero sodales faucibus.
Curabitur erat sapien, vulputate a tristique a, eleifend vel ligula.
Interdum et malesuada fames ac ante ipsum primis in faucibus.




## :question: Exercise 1 - German Translation

* Exercise: Write a prompt that generates the expected completion
* Input text: `I was enjoying the sun, but then a huge cloud came and covered the sky.`
* Expected completion: `Ich genoss die Sonne, aber dann kam eine riesige Wolke und bedeckte den Himmel.`

## :question: Exercise 2 - Classification

* Exercise: Write a prompt that generates the expected completion
* Input text: `Not much to write about here, but it does exactly what it's supposed to. filters out the pop sounds. now my recordings are much more crisp. it is one of the lowest prices pop filters on amazon so might as well buy it, they honestly work the same despite their pricing.`
* Expected completion (or similar):
  ``` 
  Positive: 0.75
  Neutral: 0.20
  Negative: 0.05
  ```

## :question: Exercise 3 - Multiple Tasks

* Exercise: Write a prompt that generates the expected completion
* Input text: `I was enjoying the sun, but then a huge cloud came and covered the sky.`
* Expected completion:
  ```
  {
      "translated": "Ich genoss die Sonne, aber dann kam eine riesige Wolke und bedeckte den Himmel.",
      "negated": "I was not enjoying the sun, and no huge cloud came and covered the sky.",
      "third_person": "She was enjoying the sun, but then a huge cloud came and covered the sky."
  }
  ```

## :question: Exercise 4 - Fashion product description

* Exercise: Write a prompt that generates the expected completion
* Input text:
  ```
  Season: Winter
  Style: Sweater
  Gender: Female
  Target group: Teenager
  Material: Cotton
  ```
* Expected completion (or similar):
  ```
  Stay warm and stylish this winter with our cozy cotton sweaters, perfect for the fashion-forward teenager. Refresh your wardrobe with the latest winter styles from our collection.
  ```

## Summary

In this lab, we used the Azure OpenAI API directly to query information about our instance of the Azure OpenAI service and to send a prompt to an OpenAI model.

## Up Next

In the next lab, we will look at using the OpenAI SDK to work with the Azure OpenAI service.

## Next Section

📣 [OpenAI Packages/Libraries](../02-OpenAIPackages/openai.ipynb)